# Standard Imports

In [769]:
# imports
import chess
import chess.svg
import chess.engine
import chess.pgn
import random
import re
import numpy as np
import pandas as pd

# Load Data

In [770]:
tahlon_pgn = open('tahlon_games.pgn')
tyler_pgn = open('tyler_games.pgn')
pd.set_option('display.max_colwidth', None)

# Fix Games

In [771]:
def pgn_to_dataframe(pgn):
    games = []
    for game in pgn:
        game = chess.pgn.read_game(pgn)
        games.append(game)
        df = pd.DataFrame(games)
    return df

tahlon_games = pgn_to_dataframe(tahlon_pgn)
tyler_games = pgn_to_dataframe(tyler_pgn)

games_list = [tahlon_games, tyler_games]

# Testing Functions

In [772]:
# random board
def random_board(max_depth=200):
    board = chess.Board()
    depth = random.randrange(0, max_depth)

    for _ in range(depth):
        all_moves = list(board.legal_moves)
        random_move = random.choice(all_moves)
        board.push(random_move)
        if board.is_game_over():
            break
    
    return board

# position score
def position_score(board, depth):
    engine = chess.engine.SimpleEngine.popen_uci('/content/stockfish')
    with engine:
        result = engine.analyse(board, chess.engine.Limit(depth=depth))
        score = result['score'].white().score()
        return score

# Transionlation Dictionary
In order for our model to interpret the chess board we have to use a python dictionary to translate chess algebraic notiation into integers.

In [773]:
algebraic_translation_dictionary = {'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,'h':8}

# Clean Data
As you can see above our data is very messy. We need to clean it up so the algebraic representation can be tranlated to the UCI representation. Therefore it should look something like this when we are done: 1. e3 d5 2. d4 Nf3 ect...

In [774]:
# def remove_initial_text_from_dataframe():
# def remove_clock_from_dataframe(df):
# def remove_trailing_dots(df): 

def rename_column(df):
    df.rename(columns = {0:'games'}, inplace = True)

rename_column(tahlon_games)
tahlon_games.games = tahlon_games.games.apply(lambda x: re.split(r"\n\n", str(x))[-1]) 
tahlon_games.games = tahlon_games.games.apply(lambda x: re.sub(r'\{[^{}]*\}', '', x))
tahlon_games.games = tahlon_games.games.apply(lambda x: re.sub(r'\.\.', ' ', x)) 
tahlon_games

,games
0,1. d4 1 . Nc6 2. Nc3 2 . b6 3. Bf4 3 . Ba6 4. e4 4 . e5 5. Bxa6 5 . exf4 6. g4 6 . Nxd4 7. Qxd4 7 . Ne7 8. Qe5 8 . d6 9. Qxf4 9 . g5 10. Qxg5 10 . Bg7 11. Qxg7 11 . Rg8 12. Qxh7 12 . f6 13. O-O-O 13 . b5 14. Re1 14 . b4 15. Na4 15 . Rb8 16. b3 16 . c5 17. Nc3 17 . bxc3 18. Ne2 18 . d5 19. Rhg1 19 . d4 20. g5 20 . fxg5 21. Rxg5 21 . Rb6 22. Rxg8+ 22 . Nxg8 23. Qxg8+ 23 . Kd7 24. Qxd8+ 24 . Kxd8 25. Rh1 25 . Rxa6 26. h4 26 . Rxa2 27. h5 27 . a5 28. Kb1 28 . Ra3 29. h6 29 . Ke8 30. h7 30 . d3 31. cxd3 31 . Rxb3+ 32. Kc2 32 . Rb7 33. h8=Q+ 33 . Kf7 34. Qh7+ 34 . Ke6 35. Qxb7 35 . a4 36. Qa7 36 . c4 37. Qxa4 37 . cxd3+ 38. Kxd3 38 . c2 39. Rc1 39 . Kd6 40. Rxc2 40 . Ke6 41. f4 1-0
1,1. e4 1 . e5 2. Bc4 2 . Qf6 3. Qh5 3 . g6 4. Bxf7+ 4 . Qxf7 5. Qxe5+ 5 . Qe6 6. Qxe6+ 6 . dxe6 7. Nf3 7 . Nf6 8. O-O 8 . Bc5 9. d4 9 . Bb6 10. e5 10 . Nd5 11. Bg5 11 . O-O 12. Na3 12 . c6 13. Rfe1 13 . Nd7 14. Be7 14 . Rf7 15. Ng5 15 . Rf4 16. Bd6 16 . Rxd4 17. Nxe6 17 . Nxe5 18. Rxe5 18 . Rd2 19. Nd4 19 . Rxd4 20. Rae1 20 . Bf5 21. Re7 21 . Nxe7 22. Bxe7 22 . Re8 23. Bg5 23 . Rde4 24. Rd1 24 . Re1+ 25. Rxe1 25 . Rxe1# 0-1
2,1. e4 1 . d5 2. d4 2 . Nc6 3. exd5 3 . Bg4 4. Qxg4 4 . Nxd4 5. Qxd4 5 . c6 6. Qf4 6 . Nf6 7. Bc4 7 . e5 8. Qxe5+ 8 . Be7 9. b3 9 . O-O 10. Nf3 10 . Bb4+ 11. Bd2 11 . cxd5 12. a3 12 . dxc4 13. axb4 13 . cxb3 14. cxb3 14 . Rc8 15. Rxa7 15 . Re8 16. Qe2 16 . Ne4 17. Rxb7 17 . Qd5 18. Nc3 18 . Qxb3 19. Nxe4 19 . Rc1+ 20. Bxc1 20 . Rc8 21. Nd6 21 . Rxc1+ 22. Kd2 22 . Rc2+ 23. Kd1 23 . Qb1# 0-1
3,1. d4 1 . d5 2. f3 1-0
4,1. d4 1 . d5 2. c4 2 . Bf5 3. Nf3 3 . f6 4. Nc3 4 . Nc6 5. e3 5 . e5 6. dxe5 6 . fxe5 7. Qa4 7 . Bb4 8. Nxe5 8 . Ne7 9. Nxc6 9 . Nxc6 10. a3 10 . Bxc3+ 11. bxc3 11 . O-O 12. cxd5 12 . Ne5 13. e4 13 . c6 14. exf5 14 . Rxf5 15. dxc6 15 . bxc6 16. Be2 16 . Qd5 17. O-O 17 . Rg5 18. g3 18 . Qe4 19. Qxe4 19 . Rd8 20. Bxg5 20 . Rd1 21. Raxd1 21 . Ng4 22. Rd8+ 22 . Kf7 23. Qe7+ 23 . Kg6 24. Rd6+ 24 . Kh5 25. Rh6+ 25 . gxh6 26. Bxg4+ 26 . Kxg4 27. Qe2+ 27 . Kh3 28. Qe6# 1-0
5,1. d4 1 . c6 2. e4 2 . d5 3. Bf4 3 . Nd7 4. Nf3 4 . Ndf6 5. Nc3 5 . Nxe4 6. Nxe4 6 . dxe4 7. Nh4 7 . f6 8. Qh5+ 8 . g6 9. Nxg6 9 . hxg6 10. Qxh8 10 . Qxd4 11. Qxg8 11 . e3 12. fxe3 12 . g5 13. Bg3 13 . Qxe3+ 14. Kd1 14 . Bg4+ 15. Be2 15 . Kd8 16. Qxf8+ 16 . Kd7 17. Qxa8 17 . Qxe2+ 18. Kc1 18 . Qxg2 19. Qxb7+ 19 . Ke6 20. Qxc6+ 20 . Qxc6 21. c3 21 . Qxh1+ 22. Kc2 22 . Qxa1 23. h4 23 . Qd1# 0-1
6,1. e4 1 . d5 2. e5 2 . f6 3. Nf3 3 . fxe5 4. Nxe5 4 . Nf6 5. d4 5 . Bg4 6. f3 6 . Bf5 7. c4 7 . Nc6 8. cxd5 8 . Nxe5 9. dxe5 9 . Nxd5 10. Be2 10 . e6 11. O-O 11 . c6 12. Nc3 12 . b5 13. Nxd5 13 . cxd5 14. f4 14 . a6 15. a4 15 . Bc5+ 16. Kh1 16 . O-O 17. axb5 17 . axb5 18. Rxa8 18 . Qxa8 19. Bxb5 19 . Qa5 20. Bd3 20 . Bxd3 21. Qxd3 21 . Qd8 22. Rf3 22 . Qh4 23. Be3 23 . Qe1+ 24. Bg1 0-1
7,1. d4 1 . c6 2. e3 2 . d5 3. h4 3 . h5 4. a4 4 . a5 5. b4 5 . Na6 6. c4 6 . Nxb4 7. f4 7 . dxc4 8. g4 8 . hxg4 9. e4 9 . e6 10. Bxc4 10 . Rxh4 11. Rxh4 11 . Qxh4+ 12. Kd2 12 . Qh2+ 13. Kc3 13 . Nf6 14. Na3 14 . Nxe4+ 15. Kb3 15 . Qg3+ 16. Bd3 16 . Nxd3 17. Ra2 17 . Nb4+ 18. Kc4 18 . Nxa2 19. d5 19 . exd5+ 20. Kd4 20 . Nac3 21. Ke5 21 . Nxd1 22. Nh3 22 . Qxh3 23. f5 23 . Nec3 24. f6 24 . gxf6+ 25. Kxf6 25 . g3 26. Nb5 26 . g2 27. Nc7+ 27 . Kd7 28. Nxa8 28 . Nxa4 29. Kxf7 29 . Qe6+ 30. Kxf8 30 . Qe8+ 31. Kg7 31 . Ne3 32. Bxe3 32 . Qxe3 33. Kg6 33 . g1=Q+ 34. Kh5 34 . Qf1 35. Kh4 35 . Qeg1 36. Kh5 36 . Qfg2 37. Kh4 37 . d4 38. Kh5 38 . d3 39. Kh4 39 . d2 40. Kh5 40 . d1=R 41. Kh4 41 . c5 42. Kh5 42 . c4 43. Kh4 43 . c3 44. Kh5 44 . c2 45. Kh4 45 . c1=N 46. Kh5 46 . Nb2 47. Nb6+ 47 . Kc7 48. Nxc8 48 . a4 49. Ne7 49 . a3 50. Nd5+ 50 . Kc6 51. Nb4+ 51 . Kb5 52. Nd3 52 . Rxd3 53. Kh4 53 . a2 54. Kh5 54 . a1=B 55. Kh4 55 . Ka4 56. Kh5 56 . b5 57. Kh4 57 . Nd1 58. Kh5 58 . b4 59. Kh4 59 . b3 60. Kh5 60 . b2 61. Kh4 61 . b1=R 62. Kh5 62 . Q2f2 63. Kh6 63 . Re3 64. Kh7 64 . Rbb3 65. Kh6 65 . Rbd3 66. Kh7 66 . Rd4 67. Kh6 67 . Re5 68. Kh7 68 . Qf6 1/2-1/2
8,1. d4 1 . e5 2. c3 2 . d6 3. d5

# Feature Engineering
We will be using deep learning to create this chess bot. In order to teach this neural network we have to transform the chess board into 3D tensors for each chess piece.
The framework for these data extraction functions was developed by Moran Reznik in this video: https://www.youtube.com/watch?v=aOwvRvTPQrs&t=371s&ab_channel=MoranReznik

In [775]:
def board_to_matrix(board):
    pieces = ['p','r','n','b','q','k']
    layers = []
    for piece in pieces:
        layers.append(create_rep_layer(board, piece))
    board_rep = np.stack(layers)
    return board_rep

def create_rep_layer(board, type):
    s = str(board)
    s = re.sub(f'[^{type}{type.upper()}]', 0, s)
    s = re.sub(f'{type}', -1, s)
    s = re.sub(f'{type.upper()}', 1, s)

    board_mat = []
    for row in s.split('\n'):
        row = row.split(' ')
        row = [int(x) for x in row]
        board_mat.append(row)
    
    return np.array(board_mat)

def move_to_matrix(move, board): #convert to algebraic to uci here
    board.push_san(move).uci()
    move = str(board.pop())

    from_output_layer = np.zeros((8,8))
    from_row = 8 - int(move[1])
    from_column = algebraic_translation_dictionary[move[0]]
    from_output_layer[from_row,from_column] = 1

    to_output_layer = np.zeroes((8,8))
    to_row = 8 - int(move[3])
    to_column = algebraic_translation_dictionary[move[2]]
    to_output_layer[to_row, to_column] = 1

def create_move_list(s):
    return re.sub('\d*\. ', '', s).split(' ')[:-1]



In [776]:
class ChessDataset(Dataset):

    def __init__(self, games):
        super(ChessDataset, self).__init__()
        self.games = games
    
    def __len__(self):
        return 40_000

    def __getitem__(self, index):
        game_i = np.random.randint(self.games.shape[0])
        random_game = tahlon_games.games.values[game_i]
        moves = create_move_list(random_game)
        game_state_i = np.random.randint(len(moves)-1)
        next_move = moves[game_state_i]
        moves = moves[:game_state_i]
        board = chess.Board()
        for move in moves:
            board.push_san(move)
        x = board_to_matrix(board)
        y = move_to_matrix(next_move, board)
        if game_state_i % 2 == 1:
            x *= -1
            return x,y

NameError: name 'Dataset' is not defined

In [ ]:
data_train = tyler_games.games
data_train_loader = DataLoader(data_train, batch_size=32, shuffle=True, drop_last=True)

AttributeError: 'DataFrame' object has no attribute 'games'

# Convolutional Neural Network
The data we are training on is a collection of _ games played by my brother Tyler. The purpose of this bot is to recreate his playing style.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class module(nn.Module):
    def __init__(self, hidden_size):
        super(module, self).__init__()
        self.conv1 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_size)
        self.bn2 = nn.BatchNorm2d(hidden_size)
        self.activation1 = nn.SELU()
        self.activation2 = nn.SELU()

    def forward(self, x):
        x_input = torch.clone(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = x + x_input
        x = self.activation2(x)
        return x
    
class ChessNet(nn.Module):
    def __init__(self, hidden_layers=4, hidden_size=200):
        super(PolicyNet, self).__init__()
        self.hidden_layers = hidden_layers
        self.input_layer = nn.Conv2d(6, hidden_size, 3, stride=1, padding=1)
        self.module_list = nn.ModuleList([module(hidden_size) for i in range(hidden_layers)])
        self.output_layer = nn.Conv2d(hidden_size, 2, 3, stride=1, padding=1)

        def foward(self, x):
            x = self.input_layer(x)
            x = F.relu(x)
            for i in range(self.hidden_layers):
                x = self.module_list[i](x)
            x = self.output_layer(x)
            return x

# Choosing Move

In [ ]:
def check_mate_single(board):
    board = board.copy()
    legal_moves = list(board.legal_moves)
    for move in legal_moves:
        board.push_uci(str(move))
        if board.is_checkmate():
            move = board.pop()
            return move
        _ = board.pop()

def distribution_over_moves(vals):
    probs = np.array(vals)
    probs = np.exp(probs)
    probs = probs / probs.sum()
    probs = probs ** 3
    probs = probs / probs.sum()
    return probs

def choose_move(board, player, color):
    legal_moves = list(board.legal_moves)
    move = check_mate_single(board)
    if move is not None:
        return move
    
    x = torch.Tensor(board_to_matrix(board)).float().to('cuda')
    if color == chess.BLACK:
        x *= -1
    x = x.unsqueeze(0)
    move = predict(x)

    vals = []
    froms = [str(legal_move)[:2] for legal_move in legal_moves]
    froms = list(set(froms))
    for from_ in froms:
        vals = move[0,:,:][8 - int(from_[1]), algebraic_translation_dictionary[from_[0]]]
        vals.append(val)

    probs = distribution_over_moves(vals)

    chosen_from = str(np.random.choice(froms, size=1, p=probs)[0][:2])
    vals = []
    for legal_move in legal_moves:
        from_ = str(legal_move)[:2]
        if from_ == chosen_from:
            to = str(legal_move)[2:]
            val = move[1,:,:][8 - int(to[1], algebraic_translation_dictionary[to[0]])]
            vals.append(val)
        else:
            vals.append(0)

    choosen_move = legal_moves[np.argmax(vals)]
    return choosen_move